In [88]:
import pandas as pd
import json

# CHUYỂN DẠNG JSON THÀNH DATAFRAME

In [89]:
# Đọc dữ liệu JSON
# df = pd.read_json('sample.json')
with open('1000_location_samples.json', encoding='utf-8') as f:
  documents = json.load(f)

# Tạo DataFrame từ danh sách các tài liệu JSON
df = pd.json_normalize(documents)

# Xóa một số cột
del df['_id.$oid']
del df['FoodyData._id.$oid']
del df['FoodyData.Latitude']
del df['FoodyData.Longitude']
del df['FoodyData.createdAt']
del df['FoodyData.domain']
del df['FoodyData.Phone']
del df['FoodyData.Promotions']
del df['FoodyData.SpecialDesc']
del df['FoodyData.Status']
del df['FoodyData.IconUrl']
del df['FoodyData.FriendAction.TotalCount']
del df['FoodyData.FriendAction.FriendActions']
del df['FoodyData.HasAlredyAddedToList']
del df['FoodyData.AdsProviders']
del df['FoodyData.CategoryGroupKey']
del df['FoodyData.Distance']
del df['FoodyData.HasDelivery']
del df['FoodyData.BankCards']
del df['FoodyData.Reviews']
del df['FoodyData.ReviewsTest']
del df['FoodyData.IsOpening']
del df['FoodyData.HasVideo']
del df['FoodyData.Floor.Id']
del df['FoodyData.Floor.Code']
del df['FoodyData.Floor.Name']
del df['FoodyData.Floor.NameEn']
del df['FoodyData.Floor.Avatar']
del df['FoodyData.Floor.Color']
del df['FoodyData.Floor.Description']
del df['FoodyData.Floor.DescriptionEn']
del df['FoodyData.BookingMobileUrl']
del df['FoodyData.PromotionPlainTitle']
del df['FoodyData.Id']
del df['FoodyData.UrlRewriteName']
del df['FoodyData.DocumentType']
del df['FoodyData.ShowInSearchResult']
del df['FoodyData.IsAd']

# Tổ chức lại field Cuisines vì có dạng array

In [90]:
# Tạo bảng mới với cột "_id" và "Cuisines"
format_cuisine_df = pd.DataFrame(columns=['_id', 'cuisine_id'])
for _, row in df.iterrows():
    _id = row['id']
    cuisines = [cuisine['Id'] for cuisine in row['FoodyData.Cuisines']]
    new_rows = pd.DataFrame({'_id': [_id] * len(cuisines), 'cuisine_id': cuisines})
    format_cuisine_df = pd.concat([format_cuisine_df, new_rows], ignore_index=True)


# In bảng mới
display(format_cuisine_df)

,_id,cuisine_id
0,200001,1
1,200002,1
2,200003,1
3,200004,1
4,200005,1
...,...,...
1138,200997,1
1139,200998,1
1140,200999,1
1141,201000,34


# Tổ chức lại field Categories vì có dạng array

In [91]:
# Tạo bảng mới với cột "_id" và "Categories"
format_category_df = pd.DataFrame(columns=['_id', 'category_id'])
for _, row in df.iterrows():
    _id = row['id']
    categories = [category['Id'] for category in row['FoodyData.Categories']]
    new_rows = pd.DataFrame({'_id': [_id] * len(categories), 'category_id': categories})
    format_category_df = pd.concat([format_category_df, new_rows], ignore_index=True)


# In bảng mới
display(format_category_df)

,_id,category_id
0,200001,3
1,200002,11
2,200003,11
3,200004,3
4,200005,11
...,...,...
1061,200996,6
1062,200997,3
1063,200998,6
1064,200999,3


# Tổ chức lại field opening time vì có dạng array

In [92]:
# Tạo bảng mới với cột "_id" và "opening_time"
format_opening_df = pd.DataFrame(columns=['_id', 'open', 'close'])
for _, row in df.iterrows():
    _id = row['id']
    
    dayOfWeeks = []
    opens = []
    closes = []

    for document in row['FoodyData.OpeningTime']:
        if document['Times']:
            dayOfWeeks.append(document['DayOfWeek'])
            opens.append(document['Times'][0]['TimeOpen'])
            closes.append(document['Times'][0]['TimeClose'])

    new_rows = pd.DataFrame({'_id': [_id] * len(dayOfWeeks), 'open': opens, 'close': closes})
    format_opening_df = pd.concat([format_opening_df, new_rows], ignore_index=True)


# In bảng mới
display(format_opening_df)

,_id,open,close
0,200001,09:00,21:00
1,200001,09:00,21:00
2,200001,09:00,21:00
3,200001,09:00,21:00
4,200001,09:00,21:00
...,...,...,...
6966,201000,09:30,17:00
6967,201000,09:30,17:00
6968,201000,09:30,17:00
6969,201000,09:30,17:00


# LẤY DANH SÁCH TẤT CẢ CÁC CUISINE

In [93]:
# Tạo danh sách cuisine từ DataFrame mới
cuisine_list = []
for cuisines in df['FoodyData.Cuisines']:
    for cuisine in cuisines:
        cuisine_list.append(cuisine['Id'])
cuisine_list = list(set(cuisine_list))

# Tạo DataFrame mới từ danh sách cuisine
cuisine_df = pd.DataFrame({'CuisineId': cuisine_list})

# Hàm trích xuất thông tin cuisine từ giá trị nguyên
def extract_info(cuisine_id):
    for cuisines in df['FoodyData.Cuisines']:
        for cuisine in cuisines:
            if cuisine['Id'] == cuisine_id:
                return cuisine['Name'], cuisine['DetailUrl']
    return None, None

# Áp dụng hàm trích xuất thông tin lên từng phần tử của cột 'CuisineId'
cuisine_df[['Name', 'DetailUrl']] = cuisine_df['CuisineId'].apply(lambda x: pd.Series(extract_info(x)))

In [94]:
display(cuisine_df)

,CuisineId,Name,DetailUrl
0,1,Món Việt,/ha-noi/dia-diem-phong-cach-viet-nam
1,2,Món Bắc,/ho-chi-minh/dia-diem-phong-cach-mien-bac
2,3,Hà Nội,/ha-noi/dia-diem-phong-cach-ha-noi
3,4,Món Trung Hoa,/ha-noi/dia-diem-phong-cach-trung-hoa
4,5,Tây Bắc,/ho-chi-minh/dia-diem-phong-cach-tay-bac
5,6,Nam Định,/ho-chi-minh/dia-diem-phong-cach-nam-dinh
6,7,Món Miền Trung,/ha-noi/dia-diem-phong-cach-mien-trung
7,8,Món Huế,/ha-noi/dia-diem-phong-cach-hue
8,12,Món Miền Nam,/ho-chi-minh/dia-diem-phong-cach-mien-nam
9,13,Miền Tây,/ho-chi-minh/dia-diem-phong-cach-mien-tay


# Lưu các Dataframe thành file Excel

In [95]:
# Drop một số cột trước khi lưu
del df['FoodyData.Categories']
del df['FoodyData.Cuisines']
del df['FoodyData.OpeningTime']


# Lưu DataFrame các địa điểm vào file Excel
df.to_excel('excel\locations.xlsx', index=False)

# Lưu DataFrame 'format_cuisine_df' vào file Excel
format_cuisine_df.to_excel('excel\\format_cuisine_df.xlsx', index=False)

# Lưu DataFrame 'format_category_df' vào file Excel
format_category_df.to_excel('excel\\format_category_df.xlsx', index=False)

# Lưu DataFrame 'format_opening_df' vào file Excel
format_opening_df.to_excel('excel\\format_opening.xlsx', index=False)

# Lưu DataFrame 'cuisine_info_df' vào file Excel
cuisine_df.to_excel('excel\\cuisines.xlsx', index=False)